In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
data = pd.read_csv('../CMU_SynTraffic_2022/CMU_SynTraffic_2022.csv')
seed = 42

To make downsampled dataset:
1. Get all real data
2. Upsample each application type with < 30k samples from smote data
3. Downsample application types with > 30k samples

all samples use seed of 42

In [15]:
data_real = data[data['Data Source'] == 'Real']
data_smote = data[data['Data Source'] == 'SMOTE']
downsampled_data = data_real

In [16]:
# temp = data_smote.loc[data_smote['Application Type'] == 'audio-streaming']

upsampling < 30k

In [17]:
for dtype in downsampled_data['Application Type'].unique():
    n = 30000 - downsampled_data['Application Type'].value_counts()[dtype]
    print(dtype, n)
    if(n<0):
        n=0
    temp = data_smote.loc[data_smote['Application Type'] == dtype].sample(n, random_state=seed)
    downsampled_data = pd.concat([downsampled_data, temp], ignore_index=True)
    #downsampled_data.concat(temp, ignore_index=True)

audio-streaming 15246
browsing -2808
chat 25049
file-transfer 20659
email 28928
p2p -18520
video-streaming 25291
voip 28535


In [18]:
downsampled_data['Application Type'].value_counts()

p2p                48520
browsing           32808
file-transfer      30000
voip               30000
email              30000
chat               30000
video-streaming    30000
audio-streaming    30000
Name: Application Type, dtype: int64

downsamples application types with >30k samples


In [19]:
downsampled_data = downsampled_data.groupby('Application Type').sample(30000, random_state=42)

In [20]:
downsampled_data['Application Type'].value_counts()

file-transfer      30000
voip               30000
email              30000
p2p                30000
chat               30000
video-streaming    30000
audio-streaming    30000
browsing           30000
Name: Application Type, dtype: int64

In [21]:
len(downsampled_data.columns)

65

remove traffic type and data source

In [22]:
downsampled_data = downsampled_data.drop(columns=['Traffic Type'])
downsampled_data = downsampled_data.drop(columns=['Data Source'])

In [25]:
downsampled_data.to_csv('./final_darknet_data.csv')